In [1]:
%load_ext autoreload
%autoreload 2
%cd ..

/home/ubuntu/Projects/scoreoperator


In [2]:
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt
from ml_collections import ConfigDict

from src.data.toys.toy_shapes import Circle
from src.models.nerualop.FNO import TimeDependentFNO1D
from src.models.diffusion.diffusion import DiffusionLoader
from src.models.diffusion.sde import BrownianSDE
from src.models.diffusion.gaussian_process import GaussianProcess
from src.utils.trainer import TrainerModule

In [3]:
circle1 = Circle(radius=1.0)
circle2 = Circle(radius=0.5)

circle1_sample = circle1.sample(n_pts=16)
circle2_sample = circle2.sample(n_pts=16)
print(f"initial conidtion shape: {circle1_sample.shape}, target shape: {circle2_sample.shape}")

initial conidtion shape: (16, 2), target shape: (16, 2)


In [4]:
config = ConfigDict()

# Gaussian process
config.gp = ConfigDict()
config.gp.input_dim = 1
config.gp.output_dim = 2
config.gp.n_sample_pts = 16
config.gp.kernel_type = "delta"
config.gp.sigma = 1.0

# SDE
config.sde = ConfigDict()
config.sde.sigma = 1.0

# FNO
config.model = ConfigDict()
config.model.output_dim = 2
config.model.lifting_dims = [8, 8, 8, 8]
config.model.max_n_modes = [8, 8, 8]
config.model.activation = "relu"
config.model.time_encoding_dim = 64
config.model.time_incrop_method = "time_modulated"

# training
config.training = ConfigDict()
config.training.seed = 42
config.training.learning_rate = 1e-3
config.training.optimizer_name = "adam"
config.training.warmup_steps = 500
config.training.train_batch_size = 64
config.training.train_num_epochs = 100
config.training.train_num_steps_per_epoch = 200
config.training.save_dir = "../pretrained/"
config.training.load_dir = "../pretrained/"

In [5]:
model = TimeDependentFNO1D(**config.model)
gp = GaussianProcess(**config.gp)
sde = BrownianSDE(**config.sde, gp=gp)
diffusion_loader = DiffusionLoader(
    sde, 
    seed=config.training.seed,
    init_cond=circle1_sample,
    batch_size=config.training.train_batch_size,
    shuffle=True)

trainer = TrainerModule(
    config=config.training,
    model=model,
    dataloader=diffusion_loader,
    enable_wandb=False
)

TypeError: BrownianSDE._transition_prob() missing 1 required positional argument: 'x0'

In [14]:
trainer.train_model()

Training: 100%|██████████| 100/100 [01:37<00:00,  1.02epoch/s, Epoch=99, eval_loss=0.0563, train_loss=0.0664]


In [1]:
def true_brownian_score(t, x, x0=circle1_sample):
    mean, std = sde._transition_prob(t, x, x0)
    return - (x - mean) / std**2

def est_brownian_score(t, x, x0=circle1_sample):
    x = jnp.expand_dims(x, axis=0)
    t = jnp.expand_dims(t, axis=0)
    score = trainer.infer_model((x, t))
    return score.squeeze()

NameError: name 'circle1_sample' is not defined